In [1]:
import pandas as pd
import matplotlib as mpl
import seaborn as sns
import sklearn

In [2]:
df = pd.read_csv("./train.csv")

Small dataset. Will definitely be doing CV.  

  Some prelim observations:  
  GarageYrBlt: why float? only year. Check that no halves, then perhaps convert to int.  
  Lots of categorical encoding to do. For the features:  
  * if no ordinality, choose one-hot (such a small dataset, so do we need to be concerned about the extra dimensionality in this case?); not target encoding, only so I don't have to think about data leakage)
  * if ordinality, choose label

  All NA have become NaN. Don't know whether that will mean different behaviour elsewhere at all  
    
Compute correlations (R-sq) *before* training to inform feature engineering. This also possible to do in CV framework?

In [3]:
df

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,142125


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [5]:
df.loc[df["GarageYrBlt"] % 1 != 0, "GarageYrBlt"].notna()

39      False
48      False
78      False
88      False
89      False
        ...  
1349    False
1407    False
1449    False
1450    False
1453    False
Name: GarageYrBlt, Length: 81, dtype: bool

In [6]:
df.loc[df["GarageYrBlt"] % 1 != 0, "GarageYrBlt"].notna().value_counts()

GarageYrBlt
False    81
Name: count, dtype: int64

In [7]:
df.loc[df["GarageYrBlt"] % 1 != 0, ["GarageYrBlt", "GarageArea"]]

,GarageYrBlt,GarageArea
39,NaN,0
48,NaN,0
78,NaN,0
88,NaN,0
89,NaN,0
...,...,...
1349,NaN,0
1407,NaN,0
1449,NaN,0
1450,NaN,0


&#8595; &#8595; scenario where df[df... will not work: only df.loc[df

In [8]:
df.loc[df["PoolQC"].notna(), "PoolQC"]

197     Ex
810     Fa
1170    Gd
1182    Ex
1298    Gd
1386    Fa
1423    Gd
Name: PoolQC, dtype: object

In [9]:
# Start by one-hot encoding MSZoning, Street, Alley
# At least a few categorical columns suitable for label encoding -- ordinality
df_onehot = pd.get_dummies(df, columns=["MSZoning", "Street", "Alley"])
df_onehot

,Id,MSSubClass,LotFrontage,LotArea,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,...,SalePrice,MSZoning_C (all),MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,Street_Grvl,Street_Pave,Alley_Grvl,Alley_Pave
0,1,60,65.0,8450,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,...,208500,False,False,False,True,False,False,True,False,False
1,2,20,80.0,9600,Reg,Lvl,AllPub,FR2,Gtl,Veenker,...,181500,False,False,False,True,False,False,True,False,False
2,3,60,68.0,11250,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,...,223500,False,False,False,True,False,False,True,False,False
3,4,70,60.0,9550,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,...,140000,False,False,False,True,False,False,True,False,False
4,5,60,84.0,14260,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,...,250000,False,False,False,True,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,62.0,7917,Reg,Lvl,AllPub,Inside,Gtl,Gilbert,...,175000,False,False,False,True,False,False,True,False,False
1456,1457,20,85.0,13175,Reg,Lvl,AllPub,Inside,Gtl,NWAmes,...,210000,False,False,False,True,False,False,True,False,False
1457,1458,70,66.0,9042,Reg,Lvl,AllPub,Inside,Gtl,Crawfor,...,266500,False,False,False,True,False,False,True,False,False
1458,1459,20,68.0,9717,Reg,Lvl,AllPub,Inside,Gtl,NAmes,...,142125,False,False,False,True,False,False,True,False,False


In [10]:
# Why in above output does MSZoning C column have "all"?
df.loc[(df["MSZoning"]=="A") | (df["MSZoning"]=="I") | (df["MSZoning"]=="RP")]

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice


Note syntax above: parentheses after df.loc[  , NOT after df[  ; the pipe

In [11]:
df_onehot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 87 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Id                1460 non-null   int64  
 1   MSSubClass        1460 non-null   int64  
 2   LotFrontage       1201 non-null   float64
 3   LotArea           1460 non-null   int64  
 4   LotShape          1460 non-null   object 
 5   LandContour       1460 non-null   object 
 6   Utilities         1460 non-null   object 
 7   LotConfig         1460 non-null   object 
 8   LandSlope         1460 non-null   object 
 9   Neighborhood      1460 non-null   object 
 10  Condition1        1460 non-null   object 
 11  Condition2        1460 non-null   object 
 12  BldgType          1460 non-null   object 
 13  HouseStyle        1460 non-null   object 
 14  OverallQual       1460 non-null   int64  
 15  OverallCond       1460 non-null   int64  
 16  YearBuilt         1460 non-null   int64  


In [21]:
from sklearn.feature_selection import r_regression
r_sq_score = r_regression(df_onehot[["LotArea"]], df_onehot["SalePrice"]) ** 2
r_sq_score 

array([0.06961332])